# Preprocessing data

Here we describe how we read in data, clean it up and provide more machine-readable column names, and separate out some columns for additional analyses.

Data is stored in:
 * /Users/aguang/CORE/tippingpoint/data
 
It consists of 3 files that have been converted from Excel (how they were provided) to CSVs in Excel:
 * TP_DV_File.csv
 * TP_Graph_Characteristics.csv
 * TP_Subject_file.csv
 
Below is brief descriptions of them (more extensive descriptions are in the Word doc)

### TP_DV_File.csv

A table containing the subject, the graph and point combination, and whether it was marked as a tipping point or not.

### TP_Graph_Characteristics.csv

These are about characteristics of the graphs. To code the characteristics 5 different reviewers coded a characteristic, if there were differences majority opinion was used if more than 3 reviewers coded the graph as the same (i.e. greater than 60% overall). Otherwise the characteristics was left blank.

### TP_Subject_file.csv

A table about the subjects, many variables here some of which are likely correlated.

In [1]:
using DataFrames
using CSV
using GLM
using Gadfly
using Statistics
using NamedArrays

DATA="/Users/aguang/CORE/tippingpoint/tippingpoint/data"

"/Users/aguang/CORE/tippingpoint/tippingpoint/data"

For each CSV file we wrote a function `clean_x` to read in the file and then rename the columns.

In [2]:
function clean_dv(filepath)
    df = CSV.File(filepath,normalizenames=true) |> DataFrame!
    names!(df, [:subj, :graphid, :tp])
end

df_dv = clean_dv(joinpath(DATA,"TP_DV_file.csv"))
size(df_dv)

(5696, 3)

For DV we additionally split out the `graphid` column into `q` (specific graph) and `pt` (specific point on the graph).

In [3]:
# adding some additional columns for category and pt
df_dv.q = [split(s)[1] for s in df_dv.graphid]
df_dv.pt = [split(s)[2] for s in df_dv.graphid]
size(df_dv)

(5696, 5)

In [4]:
function clean_gc(filepath)
    df = CSV.File(filepath,normalizenames=true) |> DataFrame!
    names!(df, [:graphid, :risingBefore, :cannotSeeAfter, :downOverall, :bellOverall, :complexOverall])
end

df_gc = clean_gc(joinpath(DATA,"TP_Graph_Characteristics.csv"))
size(df_gc)

(32, 6)

In [5]:
function clean_subject(filepath)
    df = CSV.File(filepath,normalizenames=true) |> DataFrame!
    names!(df, [:subj, :uniBrown, :expExec, :tpChange, :tpRate, :tpDir, :tpNoReturn,
            :tellMgr, :impChange, :impRise, :impFall, :impPeriodic, :numOtherTP,
            :liwcPosemo, :liwcNegemo, :liwcCause, :liwcFocusPre, :liwcFocusFut,
            :liwcRelativ, :liwcTime])
end

df_subject = clean_subject(joinpath(DATA,"TP_Subject_file.csv"))
size(df_subject)

(178, 20)

Finally, we combine all of the dataframes together into a full dataframe.

In [6]:
full_df = join(df_dv, df_gc, on= :graphid)
full_df = join(full_df, df_subject, on = :subj)
first(full_df, 6)

,subj,graphid,tp,q,pt,risingBefore,cannotSeeAfter,downOverall
,Int64,String,Int64⍰,SubStrin…,SubStrin…,Int64,Int64,Int64
1,1,Q2 A,0,Q2,A,1,0,0
2,1,Q2 B,0,Q2,B,1,0,0
3,1,Q2 C,0,Q2,C,1,1,0
4,1,Q3 A,0,Q3,A,1,0,1
5,1,Q3 B,0,Q3,B,0,0,1
6,1,Q3 C,0,Q3,C,0,0,1


┌ Warning: `T` is deprecated, use `nonmissingtype` instead.
│   caller = compacttype(::Type, ::Int64) at show.jl:39
└ @ DataFrames /Users/aguang/.julia/packages/DataFrames/Iyo5L/src/abstractdataframe/show.jl:39


Now, we save the dataframes for other notebooks to use.

In [9]:
CSV.write(joinpath(DATA,"df_dv.dat"), df_dv)
CSV.write(joinpath(DATA,"df_gc.dat"), df_gc)
CSV.write(joinpath(DATA,"df_subject.dat"), df_subject)
CSV.write(joinpath(DATA,"full_df.dat"), full_df)

"/Users/aguang/CORE/tippingpoint/tippingpoint/data/full_df.dat"